In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [2]:
import numpy as np
import tflearn

import random
import json
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [3]:
with open("intents.json") as file:
    data = json.load(file)
print(data)
print(data['intents'])

words = []
labels = []
doc_x = []
doc_y=[]

{'intents': [{'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day'], 'responses': ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], 'context': ['']}, {'tag': 'options', 'patterns': ['How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered'], 'responses': ['I can guide you through

In [4]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        doc_x.append(wrds)
        doc_y.append(intent['tag'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])


In [5]:
print(words)


['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola', 'Hello', 'Good', 'day', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you', ',', 'bye', 'Till', 'next', 'time', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Awesome', ',', 'thanks', 'Thanks', 'for', 'helping', 'me', 'How', 'you', 'could', 'help', 'me', '?', 'What', 'you', 'can', 'do', '?', 'What', 'help', 'you', 'provide', '?', 'How', 'you', 'can', 'be', 'helpful', '?', 'What', 'support', 'is', 'offered', 'How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?', 'Open', 'adverse', 'drugs', 'module', 'Give', 'me', 'a', 'list', 'of', 'drugs', 'causing', 'adverse', 'behavior', 'List', 'all', 'drugs', 'suitable', 'for', 'patient', 'with', 'adverse', 'reaction', 'Which', 'drugs', 'dont', 'have', 'adverse', 'reaction', '?', 'Open', 'blood', 'pressure', 'module', 'Task', 'related', 'to', 'blood', 'pressure', 'Blood', 'pressure', 'data', 'entry', 'I', 'want', 'to', 'log', 'blood', 'pr

In [6]:
words = [ps.stem(w.lower()) for w in words if w!= '?']
words = sorted(list(set(words)))

In [7]:
labels = sorted(labels)
training = []
output = []
out_empty = [0 for _ in range(len(labels))]


In [8]:
for x,doc in enumerate(doc_x):
    bag = []
    wrds = [ps.stem(w1) for w1 in doc]
    for w1 in words:
        if w1 in wrds:
            bag.append(1)
        else:
            bag.append(0)
    output_row = out_empty[:]
    output_row[labels.index(doc_y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [9]:
training =np.array(training)
output = np.array(output)


In [10]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

tensorflownet = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(tensorflownet, 8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
net = tflearn.regression(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [11]:
model = tflearn.DNN(net, tensorboard_dir='model_logs')

# model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.fit(  X_inputs=training, 
            Y_targets=output,
            n_epoch=1000,
            batch_size=8,
            show_metric=True
        )


Training Step: 5999  | total loss: 0.09875 | time: 0.014s
| Adam | epoch: 1000 | loss: 0.09875 - acc: 0.9955 -- iter: 40/47
Training Step: 6000  | total loss: 0.08963 | time: 0.017s
| Adam | epoch: 1000 | loss: 0.08963 - acc: 0.9960 -- iter: 47/47
--
INFO:tensorflow:e:\Chat Bot\intent-py-app\src\modelchatbot is not in all_model_checkpoint_paths. Manually adding it.


INFO:tensorflow:e:\Chat Bot\intent-py-app\src\modelchatbot.data-00000-of-00001
INFO:tensorflow:0
INFO:tensorflow:e:\Chat Bot\intent-py-app\src\modelchatbot.index
INFO:tensorflow:0
INFO:tensorflow:e:\Chat Bot\intent-py-app\src\modelchatbot.meta
INFO:tensorflow:100


In [1]:
model.save("modelchatbot.tflearn")

NameError: name 'model' is not defined

In [12]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [ps.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w==se:
                bag[i] = 1
    
    return np.array(bag)

In [13]:
def chat():
    print("Starting chat type quit to stop")
    while True:
        inp = input('You: ')
        if inp.lower() == 'quit':
            break
        results = model.predict([bag_of_words(inp, words)])
        results_index = np.argmax(results)
        tag = labels[results_index]

        for tg in data['intents']:
            if tg['tag'] == tag:
                responses = tg['responses']
        print(random.choice(responses))


In [16]:
#model.save('modelchatbot')

model.save('chatbot_model')

INFO:tensorflow:e:\Chat Bot\intent-py-app\src\chatbot_model is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:e:\Chat Bot\intent-py-app\src\chatbot_model.data-00000-of-00001
INFO:tensorflow:0
INFO:tensorflow:e:\Chat Bot\intent-py-app\src\chatbot_model.index
INFO:tensorflow:0
INFO:tensorflow:e:\Chat Bot\intent-py-app\src\chatbot_model.meta
INFO:tensorflow:100


In [ ]:
chat()